In [17]:
import sys
sys.path.insert(1, '/Users/mac/Desktop/PycharmProjects/TAADL/src')
sys.path.insert(2, '/Users/mac/Desktop/PycharmProjects/TAADL/models')

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import json

from torch import optim
from param_search import train
from network import GPCopulaRNN
from config import DATA_PATH, e_dict
from utils import transform, inv_transform

In [2]:
#TODO add mean and covariance to existing RP strategy and do the plotting

In [18]:
params = torch.load('/Users/mac/Desktop/PycharmProjects/TAADL/models/GaussCopula.pt')

In [19]:
# import feature 
prices = pd.read_csv(DATA_PATH+'/prices.csv', index_col='Date')

# preprocess weekly data
prices.index = pd.to_datetime(prices.index)
prices = prices.resample('M').last()
lret_m = np.log(prices/prices.shift(1)).fillna(0.0) # weekly return


# convert the data to torch tensor
split_idx = int(lret_m.shape[0] * 0.7)
Z_tr, Z_te = torch.Tensor(lret_m.iloc[:split_idx].values), torch.Tensor(lret_m.iloc[split_idx:].values)
X_tr, cdf = transform(Z_tr)
X_te, _   = transform(Z_te, cdf)


In [23]:
num_samples = 20
num_epochs = 250

# load parameters for model and optimizer
model = GPCopulaRNN(input_size=1, hidden_size=4, num_layers=2, rank_size=5, 
                        batch_size=3, num_asset=7, dropout=0.05, features=e_dict)
model.load_state_dict(params['net_params'])

<All keys matched successfully>

In [27]:
test(model, Z_te, cdf)

Prediction MSE : tensor([0.0116])


tensor([0.0116])

In [29]:
type(cdf)

dict